## __3.3 비지도 학습 기반 형태소 분석__

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'ML/KRembedding/'

LIB_PATH = 'lib'

%cd drive/My\ Drive/$FOLDERNAME/$LIB_PATH
!cd ../ && bash preprocess.sh dump-processed

Mounted at /content/drive
/content/drive/My Drive/ML/KRembedding/lib


In [9]:
!cd ../ && bash preprocess.sh dump-processed

download processed data...
--2021-05-25 06:32:15--  https://docs.google.com/uc?export=download&confirm=qmQw&id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG
Resolving docs.google.com (docs.google.com)... 173.194.216.100, 173.194.216.102, 173.194.216.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.216.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-24-docs.googleusercontent.com/docs/securesc/hlhq40e6ego0te6ik6vhnf5vo3ooe8au/56mltsdvg0joqb2t7pg7moq49jv97ccb/1621924275000/05170634686643261154/11210187316232883667Z/1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG?e=download [following]
--2021-05-25 06:32:16--  https://doc-04-24-docs.googleusercontent.com/docs/securesc/hlhq40e6ego0te6ik6vhnf5vo3ooe8au/56mltsdvg0joqb2t7pg7moq49jv97ccb/1621924275000/05170634686643261154/11210187316232883667Z/1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG?e=download
Resolving doc-04-24-docs.googleusercontent.com (doc-04-24-docs.googleusercontent.com)... 142.250.97.132, 26

## 3.3.1 soynlp 형태소 분석기

In [ ]:
from soynlp.word import WordExtractor

corpus_fname = "/notebooks/embedding/data/processed/processed_ratings.txt"
model_fname = "/notebooks/embedding/data/processed/soyword.model"

with open(corpus_fname, 'r') as f:
  sentences = [sent.strip() for sent in f.readlines()]
word_extractor = WordExtractor(min_frequency=100,
                               min_cohesion_forward=0.05,
                               min_right_branching_entropy=0.0)
word_extractor.train(sentences)
word_extractor.save(model_fname)

training was done. used memory 1.051 Gb


In [ ]:
import math
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

model_fname = "/notebooks/embedding/data/processed/soyword.model"

word_extractor = WordExtractor(min_frequency=100,
                               min_cohesion_forward=0.05,
                               min_right_branching_entropy=0.0)
word_extractor.load(model_fname)
scores = word_extractor.word_scores()
scores = {
    key: ( scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy) )   \
    for key in scores.keys() 
    }
tokenizer = LTokenizer(scores=scores)
print(tokenizer.tokenize('애비는 종이었다'))

all cohesion probabilities was computed. # words = 6130
all branching entropies was computed # words = 123575
all accessor variety was computed # words = 123575
['애비는', '종이었다']


`sentences` 는 네이버 영화리뷰 말뭉치의 문장들을 담고 있습니다.

In [ ]:
sentences[:10]

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ',
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.',
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.',
 '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지',
 '안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.',
 '사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화',
 '완전 감동입니다 다시봐도 감동',
 '개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음',
 '굿',
 '바보가 아니라 병 쉰 인듯']

조사와 어미, 합성어를 일부 분리해 내는 모습을 볼 수 있습니다.

In [ ]:
_ = [print(tokenizer.tokenize(line)) for line in sentences[:10]]

['어릴때', '보고', '지금', '다시봐도', '재밌어요', 'ㅋㅋ']
['디자인을', '배우', '는', '학생', '으로,', '외국', '디자이너와', '그들', '이', '일군', '전통을', '통해', '발전', '해가는', '문화', '산업이', '부러웠는데.', '사실', '우리나라', '에서도', '그', '어려운', '시절에', '끝까지', '열정', '을', '지킨', '노라노', '같은', '전통이있어', '저와', '같은', '사람들이', '꿈을', '꾸고', '이뤄나갈', '수', '있다', '는', '것에', '감사합니다.']
['폴리스스토리', '시리즈는', '1부터', '뉴까지', '버릴께', '하나', '도', '없음.', '.', '최고', '.']
['와..', '연기', '가', '진짜', '개쩔구나..', '지루', '할거라고', '생각', '했는데', '몰입', '해서', '봤다.', '.', '그래', '이런', '게', '진짜', '영화', '지']
['안개', '자욱한', '밤하늘에', '떠', '있는', '초승달', '같은', '영화', '.']
['사랑을', '해본사람이라면', '처음', '부터', '끝까지', '웃을', '수', '있는', '영화']
['완전', '감동', '입니다', '다시', '봐도', '감동']
['개들의', '전쟁', '2', '나오', '나요?', '나오면', '1빠로', '보고', '싶음']
['굿']
['바보', '가', '아니라', '병', '쉰', '인듯']


## 3.3.2 구글 센텐스피스

In [20]:
import sentencepiece as spm
train = '--input=/notebooks/embedding/data/processed/processed_wiki_ko.txt \
         --model_prefix=sentpiece \
         --vocab_size=32000 \
         --model_type=bpe \
         --character_coverage=0.9995'
spm.SentencePieceTrainer.Train(train)

In [21]:
output_fname = '/notebooks/embedding/data/processed/bert.vocab'
with open('sentpiece.vocab', 'r', encoding='utf-8') as f1, \
      open(output_fname, 'w', encoding='utf-8') as f2:
  f2.writelines("[PAD]\n[UNK]\n[CLS]\n[SEP]\n[MASK]\n")
  for line in f1:
    word = line.replace('\n', '').split('\t')[0]
    if not word or word in ["▁", "<unk>", "<s>", "</s>"]:
      continue
    if word[0] == '▁':
      word = word.replace('▁', '')
    else:
      word = '##' + word
    f2.writelines(word + "\n")

In [22]:
%cd ../
from models.bert.tokenization import FullTokenizer
%cd $LIB_PATH

vocab_fname = "/notebooks/embedding/data/processed/bert.vocab"
tokenizer = FullTokenizer(vocab_file=vocab_fname, do_lower_case=False)
tokenizer.tokenize('집에좀 가자')

/content/drive/My Drive/ML/KRembedding
/content/drive/My Drive/ML/KRembedding/lib


['집에', '##좀', '가자']

## 3.3.3 띄어쓰기 교정

In [11]:
from soyspacing.countbase import CountSpace

corpus_fname = '/notebooks/embedding/data/processed/processed_ratings.txt'
model_fname = '/notebooks/embedding/data/processed/space_correct.model'

model = CountSpace()
model.train(corpus_fname)
model.save_model(model_fname, json_format=False)

all tags length = 1166880 --> 165338, (num_doc = 199991)

In [14]:
from soyspacing.countbase import CountSpace

model_fname = '/notebooks/embedding/data/processed/space_correct.model'
model = CountSpace()
model.load_model(model_fname, json_format=False)
model.correct('어릴때보고 지금다시봐도 재밌어요')

('어릴때 보고 지금 다시봐도 재밌어요', [0, 0, 1, 0, 1, 0, 1, 0, None, 0, 1, 0, 0, 0, 1])